# Monte Carlo Parte 2

Dando seguimento em nosso estudo sobre as aplicações do método MC vamos aplicar o método em um sistema molecular. O problema que temos interesse em resolver é:

1) Calcular a o trabalho necessário para separar dois átomos de argônio de sua distância de interação no equilíbrio até uma distância no infinito. Considere a distância no infinito de 30 angstrom. Os dados do potencial de Lennard Jones que descrevem bem a interação entre os dois átomos de argônio são: $\epsilon = 0,2378$ kcal/mol e $\sigma = 3,41$ angstrom. 

- Em um primeiro momento vamos calcular o trabalho de maneira analítica e no segundo momento vamos utilizar o método MC para comparar os resultados.

###### O Potencial

- O potencial de interação não ligado entre os dois átomos de argônio é bem modelado pelo potencial de Lennard - Jones:

$$U_{LJ}(r) = 4\epsilon \Big[\Big(\frac{\sigma}{r}\Big)^{12}+\Big(\frac{\sigma}{r}\Big)^6\Big].$$

- Uma vez que os dois átomos de argônio interagem a partir de uma distância de equilíbrio será necessário saber a distância de equilíbrio da interação, portanto devemos calcular a derivada e igualar a zero.

$$\dfrac{\partial U_{LJ}}{\partial r} = 24\epsilon \Big[\Big(\frac{\sigma^6}{r^7}\Big) - 2\Big(\frac{\sigma^{12}}{r^{13}}\Big)\Big].$$

$$ 0 = 24\epsilon \Big[\Big(\frac{\sigma^6}{r^7}\Big) - 2\Big(\frac{\sigma^{12}}{r^{13}}\Big)\Big]$$

$$2\Big(\frac{\sigma^{12}}{r^{13}}\Big) = \Big(\frac{\sigma^6}{r^7}\Big)$$

$$r = 2^{1/6} \sigma.$$

Portando a distância de equilíbrio do sistema é $r_e = 2^{1/6}x3,41 = 3,827$ angstrom.

###### A Força

A relação entre força e o potencia é dada pelo negativo do gradiente.

$$F_{LJ}(r) = - \dfrac{\partial U_{LJ}}{\partial r}$$

$$F_{LJ}(r) = 24\epsilon \Big[2\Big(\frac{\sigma^{12}}{r^{13}}\Big) - \Big(\frac{\sigma^6}{r^7}\Big)\Big].$$


###### O Trabalho

O trabalho para separar os dois átomos de argônio é representado pela integral

$$W = \int_{r_e}^{r_{\infty}} F_{LJ}(r) dr.$$

- Para resolver analiticamente o trabalho (W) teremos a seguite integral.

$$W = \int_{3,827}^{30} F_{LJ}(r) dr.$$


Vamos resolver analiticamente a integral acima utilizando uma biblioteca de computação simbólica do python, chamada Sympy.


In [8]:
import sympy as sy


def trabalho_LJ(a, b, epsilon, sigma):
    '''
    A função recebe os intervalos de integração
    inicial e final, o valor das constantes 
    epsilon, sigma e retorna o valor da integral
    do trabalho.
    
    Unidades de medida
    ------------------
    epsilon(e)    [kcal/mol]
    sigma(si)     [angstrom]
    Trabalho(W)   [kcal/mol]
    '''
    e = epsilon
    si = sigma
    r = sy.Symbol('r')    
    F_LJ = 24*e*(2*(si**12/r**13) - (si**6/r**7))
    W = sy.integrate(F_LJ, (r, a, b))
    
    return round(W, 4)

print(trabalho_LJ(3.827, 30, 0.2378, 3.41))



-0.2378


Desde modo o trabalho necessário para separar dois átomos de argônio de sua posição de equilíbrio até uma certa distância adotada como infinito é -0,2378 kcal/mol.  

2) Podemos resolver esse problema utilizando a regra do trapézio composta.  

In [37]:
import numpy as np


def funcao(r, e, si):
    '''
    Recebe os valores de distância, epsilon e 
    sigma e retorna a função que deseja integrar.
    
    Unidades de medida
    ------------------
    epsilon(e)    [kcal/mol]
    sigma(si)     [angstrom]
    Trabalho(W)   [kcal/mol]
    '''
    F_LJ = 24*e*(2*(si**12/r**13) - (si**6/r**7))
    return F_LJ

def regra_trapezio_comp(x, y):
    '''
    Recebe duas listas de valores e retorna
    o valor da integral calculado pelo método 
    da regra do trapézio composta.
    '''
    regra_trap = []
    n = len(x)
    for i in range(n):
        if i == 0 or i == n-1:
            regra_trap.append(y[i])
        else:
            regra_trap.append(2*y[i])
    w = (h/2)*sum(regra_trap)

    return round(w, 4)


intervalo_ini = 3.827
intervalo_final = 30
epsilon =  0.2378
sigma = 3.41

sub_divisoes = 1000
h = (intervalo_final - intervalo_ini) / sub_divisoes

x = [i for i in np.arange(intervalo_ini, intervalo_final+0.1, h)]
y = [round(funcao(i, epsilon, sigma), 4) for i in x]
W = regra_trapezio_comp(x, y)

print(f'Valor da Integral: W = {W}')

Valor da Integral: W = -0.2376


Observe como neste caso foi necessário um conjunto de 1000 subdivisões no método dos trapézios composto para obtermos um erro relativo de 0.084 %. Assim podemos verificar que para uma melhor aproximação o método dos trapézios necessita de um conjunto maior de intervalos.

3) Agora vamos utilizar o método MC para resolver o problema. Vamos criar um programa para calcular o valor do trabalho ($W$) considerando diferentes valores de n, como uma lista de valores $[10, 10², 10³, 10⁴, 10⁵, 10⁶]$.

In [36]:
# Programa que calcula valores do trabalho
# via Monte Carlo, para separar dois átomos
# de argônio de uma distância de equilíbrio
# até o infinito.


import numpy as np
import random


def funcao(r, e, si):
    '''
    Recebe os valores de distância, epsilon e 
    sigma e retorna a função que deseja integrar.
    
    Unidades de medida
    ------------------
    epsilon(e)    [kcal/mol]
    sigma(si)     [angstrom]
    Trabalho(W)   [kcal/mol]
    '''
    F_LJ = 24*e*(2*(si**12/r**13) - (si**6/r**7))
    return F_LJ


intervalo_ini = 3.827
intervalo_final = 30
epsilon =  0.2378
sigma = 3.41

n = [10, 10**2, 10**3, 10**5, 10**6]

for n_i in n:
    f_eta = 0
    for i in range(n_i):
        eta_i = random.uniform(intervalo_ini, intervalo_final)
        f_eta += funcao(eta_i, epsilon, sigma)

    W = round((intervalo_final - intervalo_ini) * (f_eta/n_i), 4)    
    print(f'Monte Carlo: W = {W}')

Monte Carlo: W = -0.296
Monte Carlo: W = -0.181
Monte Carlo: W = -0.2438
Monte Carlo: W = -0.2372
Monte Carlo: W = -0.2382


Utilizando o método MC percebemos que o erro relativo diminui à medida que aumentamos o conjunto de números aleatórios gerados. O menor erro é de 0,16% para 1 milhão de números aleatórios gerados. Como a integral continua sendo de baixa dimensionalidade o MC necessita gerar muitos números aleatórios para resolver o problema com uma boa aproximação quando comparado com o método dos trapézios.

---
##### Questões

1) Escreva um gráfico do logaritmo do conjunto de números aleatórios gerados no Monte Carlo pelo erro relativo em relação ao trabalho exato.

2) Pelos resultados gerados com os programas, podemos afirmar que o método Monte Carlo é impreciso quando comparado com o método dos trapézios? Justifique a resposta.

---